In [1]:
import duckdb
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
dotenv_path = Path("/Users/arthurgonzaga/DataKick/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:

# Configuração das credenciais do MinIO
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
MINIO_ENDPOINT = "localhost:9000"

# Caminho do arquivo no MinIO
minio_csv_path = "s3://bronze/datadrop/kaggle/database.csv"

In [8]:

# Conectando ao DuckDB
con = duckdb.connect()

config = f"""
INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='{MINIO_ACCESS_KEY}';
SET s3_secret_access_key='{MINIO_SECRET_KEY}';
SET s3_endpoint='{MINIO_ENDPOINT}'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO
"""

# Configurando as credenciais e o endpoint do MinIO no DuckDB
con.execute(config)

In [9]:
print(config)


INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='jknVRF5166kyQRLiK7gc';
SET s3_secret_access_key='OKczJuafJxBfAQ8xn9hiHguhbQtQgmYALDwLY1Sy';
SET s3_endpoint='localhost:9000'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO



In [10]:
# Carregando o CSV do MinIO para uma tabela temporária e lendo os dados
try:
    result = con.execute(f"SELECT * FROM read_csv_auto('{minio_csv_path}')").df()
    print(result.head())  # Mostra as primeiras linhas do dataframe
except Exception as e:
    print("Erro ao acessar o MinIO:", e)

              Jogador      Time   # Nação Pos.   Idade  Min.  Gols  Assis.  \
0                Éder  Criciúma  23   ITA   FW  37-150  57.0     0       0   
1     Yannick Bolasie  Criciúma  11   COD   FW  34-325  33.0     0       0   
2       Renato Kayser  Criciúma  79   BRA   FW  28-056  85.0     1       0   
3              Arthur  Criciúma  35   BRA   FW  31-303   5.0     0       0   
4  Marquinhos Gabriel  Criciúma  10   BRA   AM  33-267  77.0     0       0   

   PB  ...  GCA   Cmp   Att  Cmp%  PrgP  Conduções  PrgC  Tent  Suc       Data  
0   0  ...  0.0  12.0  16.0  75,0   3.0       17.0   0.0   0.0  0.0 2024-04-13  
1   0  ...  0.0   4.0   5.0  80,0   0.0        6.0   3.0   5.0  3.0 2024-04-13  
2   0  ...  0.0  10.0  15.0  66,7   4.0       14.0   0.0   1.0  0.0 2024-04-13  
3   0  ...  0.0   5.0   6.0  83,3   1.0        2.0   0.0   1.0  1.0 2024-04-13  
4   0  ...  1.0  22.0  24.0  91,7   3.0       24.0   2.0   7.0  5.0 2024-04-13  

[5 rows x 33 columns]


In [11]:
# Fechar a conexão
con.close()